In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import copy

from general_ode_solver import FitzHughNagumoModel, GeneralODENumericalSolver

import os
import sys
parent_dir = os.path.abspath(os.path.join(os.path.join(os.getcwd(), ".."), ".."))
sys.path.append(parent_dir)

from data_generation.simulations.grid import Grid, fractional_transformation, logistic_transformation


## Test Usage FitzHughNagumo Model with Solver

In [12]:
fhnM = FitzHughNagumoModel(control_params=['I'])
fhnM.get_config()

{'model': 'FitzHughNagumoModel',
 'x_dim': 2,
 'control_dim': 1,
 'control_params': ['I'],
 'a': 0.7,
 'b': 1.6,
 'epsilon': 0.08,
 'I': None}

In [13]:
num_steps = 10000
control = np.tile(np.array([[0.35], [0.6]]), (num_steps,1,1)) #shape(num_steps, n_samples, control_dim)
t = 0
Z = np.array([1,0, 0, 1])
X = np.array([[1,0],[0,1]])


In [ ]:
solver = GeneralODENumericalSolver(fhnM)
solver.get_derivative(X, np.array([[0.35], [0.6]]))

array([[ 1.01666667,  0.136     ],
       [-0.4       , -0.072     ]])

In [15]:
sol_all =solver.step(X, control, 0.5, num_steps)

In [16]:
sol_steady = solver.step(X,control, 0.5, num_steps, steady_control=True)

In [18]:
np.max(sol_all-sol_steady) # Is this a problem? Should we adapt the error tolerances?

np.float64(0.02281503808509666)

## First Experiments FHN Model

In [22]:
def fhn(t, z, a, b, epsilon, I):
    v, w = z
    dvdt = v - (v**3) / 3 - w + I
    dwdt = epsilon * (v + a - b * w)
    return [dvdt, dwdt]

# Parameters
a = 0.7
b = 1.6
epsilon = 0.08

# Bifurcation parameter values for different regimes
I_values = [0.2, 0.35, 1.2]  # Monostable, Bistable, Oscillatory

# Time span for integration
t_span = (0, 100)
t_eval = np.linspace(t_span[0], t_span[1], 101)

# Initial conditions for phase space trajectories
in_con = [
    [-1.5, -1], [0.5, 0], [1.5, 1]  # Different starting points in phase space
]



In [23]:
random_array = np.random.rand(1000, 2)

In [24]:
sol_for_loop = []
for initcon in random_array:
    sol = solve_ivp(fhn, t_span, initcon, args=(a, b, epsilon, I_values[0]), t_eval=t_eval)
    sol_for_loop.append(sol)
       

In [25]:
def ode_vectorized(t, Z, a, b, epsilon, I):
    v, w = Z[:int(len(Z)/2)], Z[int(len(Z)/2):]  
    I_v = np.full_like(v, I)
    a_v = I = np.full_like(w, a)
    dvdt = v - (v**3) / 3 - w + I_v
    dwdt = epsilon * (v + a - b * w)
    return np.append(dvdt, dwdt)  # Return as a 1D array



In [26]:
initcon = random_array.transpose().flatten()
sol_vect = solve_ivp(ode_vectorized, t_span, initcon, args=(a, b, epsilon, I_values[0]), t_eval=t_eval)

In [27]:
sol_vect2 = np.zeros((100+1,) + initcon.shape)
sol_vect2[0] = initcon
initcon1 = copy.deepcopy(initcon)


for i in range(0,100):
    
    t_span1 = (i, i+1)
            
    solv = solve_ivp(ode_vectorized, t_span1, initcon1, args=(a, b, epsilon, I_values[0]), t_eval= t_span1)
            
    #Update Initial Condition
    initcon1 = solv.y[:,-1]
    print(solv.y)
    sol_vect2[i+1] = solv.y[:,-1]

[[ 0.30323654 -0.02417953]
 [ 0.02016111  0.17622175]
 [ 0.84676269  1.48972261]
 ...
 [ 0.73352868  0.74693052]
 [ 0.50255005  0.51206007]
 [ 0.36976456  0.42058428]]
[[-0.02417953 -0.80842145]
 [ 0.17622175  0.49653653]
 [ 1.48972261  1.64446694]
 ...
 [ 0.74693052  0.76005976]
 [ 0.51206007  0.50977216]
 [ 0.42058428  0.49281702]]
[[-0.80842145 -1.66289632]
 [ 0.49653653  1.03545064]
 [ 1.64446694  1.60646515]
 ...
 [ 0.76005976  0.77262292]
 [ 0.50977216  0.47964705]
 [ 0.49281702  0.58160978]]
[[-1.66289632 -1.80928344]
 [ 1.03545064  1.44948921]
 [ 1.60646515  1.53720904]
 ...
 [ 0.77262292  0.78415894]
 [ 0.47964705  0.39609583]
 [ 0.58160978  0.67007069]]
[[-1.80928344 -1.77282451]
 [ 1.44948921  1.53797829]
 [ 1.53720904  1.4636469 ]
 ...
 [ 0.78415894  0.79398674]
 [ 0.39609583  0.27768339]
 [ 0.67007069  0.74790506]]
[[-1.77282451 -1.72134804]
 [ 1.53797829  1.50112302]
 [ 1.4636469   1.38979172]
 ...
 [ 0.79398674  0.80110676]
 [ 0.27768339  0.16587255]
 [ 0.74790506  0.812

In [28]:
sol_vect.y

array([[ 0.30323654, -0.02411829, -0.80840906, ..., -1.29439228,
        -1.29439228, -1.29439228],
       [ 0.02016111,  0.17621643,  0.49654297, ..., -1.29439214,
        -1.29439206, -1.29439205],
       [ 0.84676269,  1.48973945,  1.64426795, ..., -1.29439222,
        -1.29439218, -1.29439218],
       ...,
       [ 0.73352868,  0.74693052,  0.76005976, ..., -0.37149517,
        -0.37149518, -0.37149518],
       [ 0.50255005,  0.51205987,  0.50977223, ..., -0.37149517,
        -0.37149517, -0.37149517],
       [ 0.36976456,  0.42058382,  0.49281723, ..., -0.37149518,
        -0.37149519, -0.3714952 ]])

In [29]:
max(sol_vect2.transpose()[0]-sol_vect.y[0])

np.float64(0.0011341972629452801)

In [30]:
sol_for_loop[0].y

array([[ 0.30323654, -0.02428705, -0.80862727, -1.66310966, -1.80987373,
        -1.77278836, -1.72126791, -1.67201934, -1.62541891, -1.58289029,
        -1.54423003, -1.50850792, -1.47701121, -1.44829728, -1.422924  ,
        -1.40110194, -1.38122343, -1.36511434, -1.35115849, -1.33891106,
        -1.32953558, -1.32183912, -1.31512445, -1.30993208, -1.30632181,
        -1.30337979, -1.30080014, -1.29880698, -1.29770619, -1.29694985,
        -1.29627209, -1.29559454, -1.29501811, -1.29480098, -1.29486143,
        -1.29494446, -1.29491711, -1.29473065, -1.29442063, -1.29410681,
        -1.2939932 , -1.29426176, -1.29466739, -1.2949547 , -1.29498275,
        -1.29472278, -1.29425817, -1.29378446, -1.29360937, -1.29394421,
        -1.29437638, -1.29469277, -1.294784  , -1.29462768, -1.29428843,
        -1.29391787, -1.29375459, -1.29393351, -1.29409434, -1.29421425,
        -1.29429971, -1.29435585, -1.29438644, -1.29439387, -1.29437921,
        -1.29433295, -1.29407385, -1.29375328, -1.2

In [31]:
max(sol_vect.y[-1]-sol_for_loop[-1].y[-1])

np.float64(0.00026417812492129933)

In [32]:
sol_vect.y[int(len(sol_vect.y)/2)]

array([ 0.69621391,  0.67719251,  0.62017259,  0.50041847,  0.35963989,
        0.23420864,  0.12748119,  0.03740086, -0.03826022, -0.10150419,
       -0.15409648, -0.19758633, -0.2333309 , -0.26251679, -0.28617956,
       -0.30522039, -0.32042026, -0.33245272, -0.34189509, -0.3492386 ,
       -0.3548976 , -0.35921825, -0.3624866 , -0.36493631, -0.3667559 ,
       -0.3680957 , -0.36907382, -0.36978217, -0.37029103, -0.37065396,
       -0.37091116, -0.37109198, -0.37121816, -0.37130605, -0.37136672,
       -0.3714081 , -0.37143657, -0.37145604, -0.37146899, -0.37147771,
       -0.37148376, -0.37148773, -0.3714902 , -0.37149185, -0.37149316,
       -0.37149407, -0.37149452, -0.37149461, -0.37149458, -0.37149474,
       -0.37149497, -0.37149516, -0.37149525, -0.37149524, -0.37149515,
       -0.37149502, -0.37149494, -0.37149498, -0.37149503, -0.37149507,
       -0.3714951 , -0.37149512, -0.37149514, -0.37149515, -0.37149515,
       -0.37149516, -0.37149516, -0.37149517, -0.37149517, -0.37